In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.worker_pools

In [ ]:
assert len(high_client.worker_pools.get_all()) == 1

In [ ]:
external_registry = test_settings.get("external_registry", default="docker.io")
external_registry

In [ ]:
worker_docker_tag = f"openmined/bigquery:{sy.__version__}"
worker_docker_tag

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=f"{external_registry}/{worker_docker_tag}")
docker_config

In [ ]:
result = high_client.api.services.worker_image.submit(worker_config=docker_config)
result

In [ ]:
worker_image = high_client.images.get_all()[1]
worker_image

In [ ]:
result = high_client.api.services.image_registry.add(external_registry)
result

In [ ]:
worker_pool_name = "bigquery-pool"

In [ ]:
result = high_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
)
result

In [ ]:
# result = high_client.worker_pools.scale(number=5, pool_name=worker_pool_name)
# result

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

In [ ]:
high_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert len(high_client.api.services.user.get_all()) == 2

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery

""".strip()
worker_dockerfile

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
server.land()

In [ ]:
# !echo "$worker_dockerfile" | docker build -t $docker_tag -q -

In [ ]:
# !docker image ls | grep bigquery